In [1]:
!pip install pgmpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 8.8 MB/s eta 0:00:009.7 MB/s eta 0:00:01
  Using cached tqdm-4.66.1-py3-none-any.whl (78 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)


In [ ]:
graph:
  - from: SMK
    to: LC
  - from: FH
    to: LC
  - from: LC
    to: PN
  - from: SMK
    to: PN

In [2]:
from pgmpy.base import DAG

dag = DAG([('SMK', 'LC'), ('FH', 'LC'), ('LC', 'PN'), ('SMK', 'PN')])

In [3]:
dag.local_independencies(['SMK', 'LC', 'FH', 'PN'])

(SMK ⟂ FH)
(FH ⟂ SMK)
(PN ⟂ FH | SMK, LC)

In [19]:
import yaml
with open('../data/fire_barn.yml') as file:
    data = yaml.safe_load(file)

In [25]:
all_variables = list(set([data['graph'][i]['from'] for i in range(len(data['graph']))] + [data['graph'][i]['to'] for i in range(len(data['graph']))]))
dag = DAG([(data['graph'][i]['from'], data['graph'][i]['to']) for i in range(len(data['graph']))])
indep = dag.get_independencies()

In [60]:
indep.__dict__['independencies'][0]

(D ⟂ F, SF, M, L | S)

In [43]:
all_variables

['D', 'L', 'SF', 'F', 'WS', 'BB', 'S', 'M']

In [56]:
[1,2] == sorted([2,1])


True

In [59]:
import random

def sample_CI_statement(all_variables):
    # Sample the first variable
    var1 = random.choice(all_variables)
    all_variables.remove(var1)

    # Sample the second variable
    var2 = random.choice(all_variables)
    all_variables.remove(var2)

    # Sample a random number of elements from the remaining ones
    num_elements = random.randint(0, len(all_variables))
    conditioning_set = random.sample(all_variables, num_elements)

    return var1, var2, conditioning_set

def graph_CI_test(var1, var2, conditioning_set, dag):
    all_CI = dag.get_independencies().__dict__['independencies']
    
    for CI in all_CI:
        if var1 in list(CI.__dict__['event1']):
            if var2 in list(CI.__dict__['event2']):
                if sorted(conditioning_set) == sorted(list(CI.__dict__['event3'])):
                    return True
    return False

In [ ]:
graph_CI_test('D', 'F', ['S'])